In [1]:
import time
import pandas as pd
import torch
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
torch.cuda.is_available()

1
NVIDIA GeForce GTX 850M


True

In [2]:
!pip install transformers
!pip3 install emoji

^C


In [3]:
from transformers import AutoTokenizer, AutoModel

import torch

inputlength = 128


class BERTTweet:

    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False, normalization=True)
        self.bertweet = AutoModel.from_pretrained("vinai/bertweet-base").to('cuda:0')

    def tweet_to_vec_string(self, tweet):
        vec = self.tweet_to_vec(tweet)
        return torch.flatten(vec).tolist()

    def tweet_to_vec(self, tweet):
        tokens = self.tokenizer.encode(tweet)

        if len(tokens) > inputlength:
            tokens = tokens[0:inputlength]

        input_ids = torch.tensor([tokens]).to('cuda:0')

        with torch.no_grad():
            features = self.bertweet(input_ids)

        return features.pooler_output


C:\ProgramData\Anaconda3\envs\tensorflow1\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\envs\tensorflow1\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\envs\tensorflow1\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\envs\tensorflow1\lib\site-pack

ValueError: tensorflow.__spec__ is None

In [ ]:
import time
import pandas as pd


class BERTCalculator:

    def __init__(self):
        self.tweet_bert = BERTTweet()

    def pre_processing(self, tweet):
        tweet = tweet.replace('\n', '')
        tweet = tweet.replace('\t', '')
        tweet = tweet.replace('\r', '')
        return tweet

    def pipeline(self, index, row, data_set):
        tweet = row["text"]

        tweet = self.pre_processing(tweet)

        vec = self.tweet_bert.tweet_to_vec_string(tweet)
        data_set._set_value(index, 'bert', vec)


def vectorize_tweets(data):
    print("VectorizeBert started")
    mechanism = BERTCalculator()

    data["bert"] = [[]] * len(data)
    t0 = time.time()

    for index, row in data.iterrows():

        try:
            mechanism.pipeline(index, row, data)

        except IndexError as err:
            print(err)

        except Exception as err:
            print(row["text"])
            raise err

        if index % 1000 == 0:
            print("index", index, " time: ", (time.time() - t0))

    print("VectorizeBert done")

    return data


In [ ]:
root_path = "/content/drive/MyDrive/Dev/"
file = "03 2018_en"
try:
    data = pd.read_csv(root_path + file + ".csv", sep=";")
except:
    data = pd.read_csv(root_path + file + ".csv", lineterminator="\n")


print("Loaded", file, " ", data.shape)

data = vectorize_tweets(data)
data.to_csv(path_or_buf=root_path + file + "_bert.csv")


In [5]:
bert = ""
for i in range(0,768):
    bert += ",bert" +str(i)

bert += ","
time_interval = "3H"
potential_path = "../Data/2018-Weighted/cache/" + "TWEETS" + "(" + time_interval + ")" + ".csv"

with open(potential_path, 'r') as infile, open(potential_path, 'w') as outfile:
    data = infile.read()
    data = data.replace('"', '')
    data = data.replace('[', '')
    data = data.replace(']', '')
    data = data.replace(',bert,', bert)

    outfile.write(data)
